In [144]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.base import BaseEstimator
from sklearn.base import RegressorMixin
from sklearn.base import clone
from sklearn.base import TransformerMixin

## Import the data

In [158]:
df=pd.read_csv('final_data_Copy.csv')
print(df.iloc[0].SalePrice_K)
df=df[['m2', 'nb_rooms','nb_bedrooms','nb_bath','apartment_floor','building_floor',
        'Cellar','Parking', 'balc_patio','Renovated', 'Ground_floor',
       'Last_floor', 'district','building_height','SalePrice_K']]
df.head()

5.794841205603168


,m2,nb_rooms,nb_bedrooms,nb_bath,apartment_floor,building_floor,Cellar,Parking,balc_patio,Renovated,Ground_floor,Last_floor,district,building_height,SalePrice_K
0,3.496508,1,0,1,1,14,0,1,0,0,0,0,19e,Tower,5.794841
1,4.804021,5,4,1,2,4,1,1,Terrasse,0,0,0,4e,Average,7.570443
2,4.219508,4,3,1,0,3,0,0,Terrasse,0,1,0,10e,Small,6.721426
3,3.583519,2,1,1,2,0,1,0,0,0,0,0,18e,Average,6.261492
4,3.465736,1,0,1,8,9,1,0,0,0,0,0,15e,Average,5.926926


## Preprocessing

In [152]:
features=[np.exp(3.49650756146648), 1, 0, 1, 1,14 ,'No', 'Yes', 'Nothing', 'No', 'No', 'No', '19e']
features

[32.999999999999986,
 1,
 0,
 1,
 1,
 14,
 'No',
 'Yes',
 'Nothing',
 'No',
 'No',
 'No',
 '19e']

In [153]:
def height(x):
    if x ==0:
        return 'Average'
    elif x<4:
        return 'Small'
    elif x<=9:
        return 'Average'
    elif x<=12:
        return 'High'
    else:
        return 'Tower'

In [154]:
def preprocessing(features):
    features[0]=np.log(features[0])
    new_features = [0 if x=='No' else 1 if x=="Yes" else x for x in features]
    new_features.append(height(new_features[5]))
    if new_features[8]=='Nothing':
        new_features[8]=0
    df=pd.DataFrame(new_features)
    return new_features
    

In [155]:
my_features=preprocessing(features)
len(my_features)

14

In [156]:
data_in=pd.DataFrame(np.array(my_features).reshape(1,-1),columns=['m2', 'nb_rooms','nb_bedrooms','nb_bath','apartment_floor','building_floor','Cellar','Parking', 'balc_patio','Renovated', 'Ground_floor','Last_floor', 'district','building_height'])

## Load the encoder

In [157]:
import pickle
with open('encoder.p', 'rb') as pickled:
    data = pickle.load(pickled)
    encoder=data['encoder']
encoder
        

FileNotFoundError: [Errno 2] No such file or directory: 'encoder.p'

In [140]:
data_in_enc=pd.DataFrame(encoder.transform(np.array(data_in[['balc_patio','district','building_height']]).reshape(1,-1)))
data_in_enc.columns=encoder.get_feature_names(['balc_patio','district','building_height'])
data_in.drop(['balc_patio','district','building_height'],axis=1,inplace=True)
data_out=pd.concat([data_in,data_in_enc],axis=1)

In [141]:
data_out

,m2,nb_rooms,nb_bedrooms,nb_bath,apartment_floor,building_floor,Cellar,Parking,Renovated,Ground_floor,...,district_4e,district_5e,district_6e,district_7e,district_8e,district_9e,building_height_Average,building_height_High,building_height_Small,building_height_Tower
0,3.49650756146648,1,0,1,1,14,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [142]:
import pickle
file_name = "FlaskAPI/models/model_file.p"
with open(file_name, 'rb') as pickled:
       data = pickle.load(pickled)
       model = data['model']
np.exp(model.predict(data_out))

C:\Users\gaeta\anaconda3\envs\fastai-cpu-v0.7\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.22.1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\gaeta\anaconda3\envs\fastai-cpu-v0.7\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.22.1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


array([328.6])